<a href="https://colab.research.google.com/github/Motolover/Web_analytics/blob/main/Export_products_to_avito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JOTTOY

## Библиотеки

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import lxml
import json
import numpy as np
import html
import time
import os
import sys
import lxml

In [ ]:
from google.colab import files
from lxml import etree

In [ ]:
!pip install gspread_formatting

In [ ]:
import gspread
import oauth2client
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting.dataframe import format_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
from google.colab import auth

In [ ]:
import yaml

In [ ]:
# Подключение к Google Таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive']

# Авторизация пользователя с помощью JSON-ключа
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/My Drive/able-engine-276816-f1ea18570bfa.json', scope)
client = gspread.authorize(credentials)

In [ ]:
gs = gspread.authorize(credentials)
worksheet = gs.open('joytoy_parsing_table').worksheet('Import Matrix 2.0')
print('таблица подключена')

таблица подключена


In [ ]:
rows = worksheet.get_all_values()
joytoy_wh_df = pd.DataFrame.from_records(rows)
joytoy_wh_df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

##**Import XLSX to Avito**

In [ ]:
joytoy_wh_df

,Franchise,SKU (new),SKU (orig),Avito_ID,Category,Name,Short_name,Release,Preorder,Link,...,OMW,Sales,Cost_price,Reasonable_price,Price,Price_WAKO,Articul,,,
0,JoyToy Warhammer 40K,6973130371286,6973130371286,,Ultramarines,Ultramarines Incursors Sergeant Romulo,JoyToy Ultramarines Incursors Sergeant Romulo,01.09.2021,,https://www.joytoy.com/products/ultramarines-i...,...,0,9,0,0,2000,2000,JT1,,,12
1,JoyToy Warhammer 40K,6973130371287,6973130371286,,Ultramarines,Ultramarines Incursors Brother Varron,JoyToy Ultramarines Incursors Brother Varron,01.09.2021,,https://www.joytoy.com/products/ultramarines-i...,...,0,9,0,0,2000,"2,000",,,,12
2,JoyToy Warhammer 40K,6973130371289,6973130371286,,Ultramarines,Ultramarines Incursors Brother Aurelian,JoyToy Ultramarines Incursors Brother Aurelian,01.09.2021,,https://www.joytoy.com/products/ultramarines-i...,...,0,9,0,0,2000,"2,000",,,,12
3,JoyToy Warhammer 40K,6973130371288,6973130371286,,Ultramarines,Ultramarines Incursors Brother Remus,JoyToy Ultramarines Incursors Brother Remus,01.09.2021,,https://www.joytoy.com/products/ultramarines-i...,...,0,9,0,0,2000,"2,000",,,,12
4,JoyToy Warhammer 40K,6973130371293,6973130371293,2657627416,Ultramarines,Ultramarines Infiltrators Brother Pullo,JoyToy Ultramarines Infiltrators Brother Pullo,01.09.2021,,https://www.joytoy.com/products/ultramarines-i...,...,0,14,0,0,2000,"2,000",,,,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,LEGO,21062,21062,,,Lego 21062 Trevi Fountain,Lego 21062 Trevi Fountain,,Предзаказ!,https://www.lego.com/en-us/product/trevi-fount...,...,0,0,0,0,"16,000","16,000",,,,
1068,LEGO,31215,31215,,,Lego 31215 Vincent van Gogh – Sunflowers,Lego 31215 Vincent van Gogh – Sunflowers,,Предзаказ!,https://www.lego.com//en-us/product/vincent-va...,...,0,0,0,0,"22,000","22,000",,,,
1069,LEGO,21355,21355,,,Lego 21355 The Evolution of STEM,Lego 21355 The Evolution of STEM,,Предзаказ!,https://www.lego.com//en-us/product/the-evolut...,...,0,0,0,0,"9,000","9,000",,,,
1070,LEGO,10362,10362,,,Lego 10362 French Café,Lego 10362 French Café,,Предзаказ!,https://www.lego.com//en-us/product/icons-1036...,...,0,0,0,0,"9,000","9,000",,,,


In [ ]:
## Фильтруем позици БЕЗ Avito_ID
#joytoy_wh_df = joytoy_wh_df[joytoy_wh_df['Avito_ID'].isin([''])]

In [ ]:
#Создаем ключевые столбцы из парсинга датафрейма
avit_df = joytoy_wh_df[['Franchise', 'SKU (new)', 'Name', 'Short_name', 'Link', 'URL', 'Stock', 'Price', 'Scale', 'Preorder', 'Applicable Age', 'Scale', 'Articul', 'Avito_ID']]

In [ ]:
## Фильтруем только джойтой
#avit_df = avit_df[avit_df['Franchise'].isin(["JoyToy Warhammer 40K", "JoyToy Warhammer 30K"])]

In [ ]:
#value_list = ["6973130376342", "6973130377882", "6973130378865", "6973130379787", "6973130376137", "6973130376175", "6927054400218"]

# Фильтруем актуальные позиции
#avit_df = avit_df[~avit_df['SKU (new)'].isin(value_list)]


**Задаем столбцы со статичными значениями**

In [ ]:
Description_string_stock = """В нaличии. Hовый, запечатанный.

{Franchise} {Name}

Oписание модeли:
· Маcштаб пpoдуктa: 1:18
· Maтериал: АБС-ПВX
· Применимый вoзраcт: {Applicable Age}
· UРC : {SKU (new)}

Oчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.

❤️ Добавь это объявление к себе в избранное

✅ В профиле есть еще очень много интересного!)

💬 Подписывайся на наши соцсети!

Если интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!

# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, Черные Храмовники, Dark Angels, Темные Ангелы, Grey Knights, Серые Рыцари, Imperial Fists, Имперские Кулаки, Iron Hands, Железные Руки, Necrons, Некроны, Orkcs, Орки, Raven Guards, Гвардия Ворона, Salamanders, Саламандры, Sons of Horus, Сыны Хоруса, T’au Empire, Империя Тау, White Consuls, Белые Консулы, White Scars, Белые Шрамы, Tyranids, Тираниды, Ваха, Коллекционные фигурки, Миниатюры, 1/18, 1/6, League of Legends"""

Description_string_stock

'В нaличии. Hовый, запечатанный.\n\n{Franchise} {Name}\n\nOписание модeли:\n· Маcштаб пpoдуктa: 1:18\n· Maтериал: АБС-ПВX\n· Применимый вoзраcт: {Applicable Age}\n· UРC : {SKU (new)}\n\nOчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.\n\n❤️ Добавь это объявление к себе в избранное\n\n✅ В профиле есть еще очень много интересного!)\n\n💬 Подписывайся на наши соцсети!\n\nЕсли интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!\n\n# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, 

In [ ]:
Description_string_preorder = """{Preorder}.

{Franchise} {Name}

Oписание модeли:
· Маcштаб пpoдуктa: 1:18
· Maтериал: АБС-ПВX
· Применимый вoзраcт: {Applicable Age}
· UРC : {SKU (new)}

Oчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.

❤️ Добавь это объявление к себе в избранное

✅ В профиле есть еще очень много интересного!)

💬 Подписывайся на наши соцсети!

Если интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!

# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, Черные Храмовники, Dark Angels, Темные Ангелы, Grey Knights, Серые Рыцари, Imperial Fists, Имперские Кулаки, Iron Hands, Железные Руки, Necrons, Некроны, Orkcs, Орки, Raven Guards, Гвардия Ворона, Salamanders, Саламандры, Sons of Horus, Сыны Хоруса, T’au Empire, Империя Тау, White Consuls, Белые Консулы, White Scars, Белые Шрамы, Tyranids, Тираниды, Ваха, Коллекционные фигурки, Миниатюры, 1/18, 1/6, League of Legends"""

Description_string_preorder

'{Preorder}.\n\n{Franchise} {Name}\n\nOписание модeли:\n· Маcштаб пpoдуктa: 1:18\n· Maтериал: АБС-ПВX\n· Применимый вoзраcт: {Applicable Age}\n· UРC : {SKU (new)}\n\nOчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.\n\n❤️ Добавь это объявление к себе в избранное\n\n✅ В профиле есть еще очень много интересного!)\n\n💬 Подписывайся на наши соцсети!\n\nЕсли интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!\n\n# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, Черные Храмовники, D

In [ ]:
Description_string_nonstok = """Наличие данной модели необходимо уточнить в ЛС.

{Franchise} {Name}

Oписание модeли:
· Маcштаб пpoдуктa: 1:18
· Maтериал: АБС-ПВX
· Применимый вoзраcт: {Applicable Age}
· UРC : {SKU (new)}

Oчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.

❤️ Добавь это объявление к себе в избранное

✅ В профиле есть еще очень много интересного!)

💬 Подписывайся на наши соцсети!

Если интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!

# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, Черные Храмовники, Dark Angels, Темные Ангелы, Grey Knights, Серые Рыцари, Imperial Fists, Имперские Кулаки, Iron Hands, Железные Руки, Necrons, Некроны, Orkcs, Орки, Raven Guards, Гвардия Ворона, Salamanders, Саламандры, Sons of Horus, Сыны Хоруса, T’au Empire, Империя Тау, White Consuls, Белые Консулы, White Scars, Белые Шрамы, Tyranids, Тираниды, Ваха, Коллекционные фигурки, Миниатюры, 1/18, 1/6, League of Legends"""

Description_string_nonstok

'Наличие данной модели необходимо уточнить в ЛС.\n\n{Franchise} {Name}\n\nOписание модeли:\n· Маcштаб пpoдуктa: 1:18\n· Maтериал: АБС-ПВX\n· Применимый вoзраcт: {Applicable Age}\n· UРC : {SKU (new)}\n\nOчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.\n\n❤️ Добавь это объявление к себе в избранное\n\n✅ В профиле есть еще очень много интересного!)\n\n💬 Подписывайся на наши соцсети!\n\nЕсли интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!\n\n# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, 

In [ ]:
def replace_placeholders(row):
    # Определяем шаблон в зависимости от нескольких условий
    if row['Stock'] > '0':
        Description = Description_string_stock
    elif row['Stock'] == '0' and row['Preorder'].strip() != '':
        Description = Description_string_preorder
    else:
        Description = Description_string_nonstok

    # Подставляем значения в выбранный шаблон
    Description = Description.replace('{Preorder}', row['Preorder'])
    Description = Description.replace('{Name}', row['Name'])
    Description = Description.replace('{Franchise}', row['Franchise'])
    #description = description.replace('{Scale}', row['Scale'])
    Description = Description.replace('{SKU (new)}', str(row['SKU (new)']))
    Description = Description.replace('{Applicable Age}', row['Applicable Age'])

    return Description

# Применение функции ко всем строкам DataFrame
avit_df['Description'] = avit_df.apply(replace_placeholders, axis=1)

# Применение функции с использованием .loc[] для присвоения значений
#avit_df.loc[:, 'Description'] = avit_df.apply(replace_placeholders, axis=1)

<ipython-input-18-d6de5e98c203>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avit_df['Description'] = avit_df.apply(replace_placeholders, axis=1)


In [ ]:
# Добавляем столбец с шаблонным описанием
#avit_df['AvitoId'] = ''
avit_df['AdStatus'] = 'Free'
avit_df['Category'] = 'Коллекционирование'
avit_df['GoodsType'] = 'Модели'
avit_df['VehicleType'] = ''
avit_df['Address'] = 'Москва, метро Улица Скобелевская'
avit_df['Condition'] = 'Новое'
avit_df['AdType'] = 'Продаю своё'
avit_df['ContactPhone'] = '+7 985 182-56-36'
avit_df['DateBegin'] = ''
avit_df['DateEnd'] = ''
avit_df['AllowEmail'] = ''
avit_df['ManagerName'] = ''
avit_df['ImageNames'] = ''

<ipython-input-19-e6bf9946c490>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avit_df['AdStatus'] = 'Free'
<ipython-input-19-e6bf9946c490>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avit_df['Category'] = 'Коллекционирование'
<ipython-input-19-e6bf9946c490>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [ ]:
# Выполняем переименование столбцов под требования шаблона
new_column_name = ['Id', 'Title', 'ImageUrls', 'Stock', 'AvitoId']
old_column_name = ['SKU (new)', 'Short_name', 'URL', 'В наличии', 'Avito_ID'] # имя столбца, которое вы хотите изменить
avit_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)

<ipython-input-20-dcddd53a7dd5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avit_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)


In [ ]:
# Чистим лишние ссылки в фото. До 5 фотограций максимум
# Функция для оставления части строки по количеству вхождения символа
def keep_part_by_char_count(string, char, count):
    # Ищем индексы вхождений символа
    indices = [i for i, c in enumerate(string) if c == char]
    if len(indices) < count:
        return string
    else:
        return string[:indices[count-1]+1]

# Применяем функцию к определенным строкам в столбце 'text'
char = '|'
count = 10
avit_df['ImageUrls'] = avit_df['ImageUrls'].apply(lambda x: keep_part_by_char_count(x, char, count))

<ipython-input-21-ae577738ad01>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avit_df['ImageUrls'] = avit_df['ImageUrls'].apply(lambda x: keep_part_by_char_count(x, char, count))


In [ ]:
# Получаем таблицу с предзаказами
avit_df_1 = avit_df[((avit_df['Stock'] == '0') & (avit_df['Preorder'] != ''))]
# Получаем ассортимент в наличии
avit_df_2 = avit_df[((avit_df['Stock'] != '0') & (avit_df['Preorder'] == ''))]

In [ ]:
# Аналог UNION - объединение всех строк
union_df = pd.concat([avit_df_1, avit_df_2], ignore_index=True).drop_duplicates()

In [ ]:
union_df

,Franchise,Id,Name,Title,Link,ImageUrls,Stock,Price,Scale,Preorder,...,VehicleType,Address,Condition,AdType,ContactPhone,DateBegin,DateEnd,AllowEmail,ManagerName,ImageNames
0,JoyToy Warhammer 40K,6973130372467,Ultramarines Venerable Dreadnought,JoyToy Ultramarines Venerable Dreadnought,https://www.joytoy.com/products/ultramarines-v...,https://img.joytoy.com/u_file/2206/28/photo/01...,0,7000,1:18,ПРЕДЗАКАЗ. Релиз июль 2025!,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
1,JoyToy Warhammer 40K,6973130372634,Blood Angels Furioso Dreadnought Brother Samel,JoyToy Blood Angels Furioso Dreadnought Samel,https://www.joytoy.com/products/blood-angels-f...,https://img.joytoy.com/u_file/2206/22/photo/01...,0,8000,1:18,В пути!,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
2,JoyToy Warhammer 40K,6973130372641,Death Korps of Krieg Veteran Squad Guardsman D...,JoyToy Death Korps of Krieg Demolitions Specia...,https://www.joytoy.com/products/death-korps-of...,https://img.joytoy.com/u_file/2206/22/photo/A0...,0,2500,1:18,В пути!,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
3,JoyToy Warhammer 40K,6973130372658,Death Korps of Krieg Veteran Squad Guardsman,JoyToy Death Korp of Krieg Veteran Squad Guard...,https://www.joytoy.com/products/death-korps-of...,https://img.joytoy.com/u_file/2206/22/photo/B0...,0,2500,1:18,В пути!,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
4,JoyToy Warhammer 40K,6973130372665,Death Korps of Krieg Veteran Squad Sergeant,JoyToy Death Korps of Krieg Veteran Squad Serg...,https://www.joytoy.com/products/death-korps-of...,https://img.joytoy.com/u_file/2206/24/photo/C0...,0,2500,1:18,В пути!,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,LEGO,76434,Lego 76434 Aragog in the Forbidden Forest,Lego 76434 Aragog in the Forbidden Forest,https://www.lego.com//en-us/product/aragog-in-...,https://www.lego.com/cdn/cs/set/assets/blt6ffb...,1,2200,-,,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
171,LEGO,21344,Lego 21344 The Orient Express Train,Lego 21344 The Orient Express Train,https://www.lego.com//en-us/product/the-orient...,https://www.lego.com/cdn/cs/set/assets/blt4c79...,1,28800,-,,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
172,LEGO,21350,Lego 21350 Jaws,Lego 21350 Jaws,https://www.lego.com/en-us/product/jaws-21350,https://www.lego.com/cdn/cs/set/assets/bltf513...,1,"20,000",-,,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,
173,LEGO,75429,Lego 75429 AT-AT Driver Helmet,Lego 75429 AT-AT Driver Helmet,https://www.lego.com//en-us/product/at-at-driv...,https://www.lego.com/cdn/cs/set/assets/bltdd27...,1,"8,500",-,,...,,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,


In [ ]:
# Выводим столбцы в порядке шаблона
avit_df_final = union_df[['Id', 'AvitoId', 'AdStatus', 'Category', 'GoodsType', 'VehicleType', 'Address', 'Title', 'Description', 'Condition', 'Price', 'AdType', 'DateBegin', 'DateEnd', 'AllowEmail', 'ManagerName', 'ContactPhone', 'ImageNames', 'ImageUrls', 'Stock']]
avit_df_final

,Id,AvitoId,AdStatus,Category,GoodsType,VehicleType,Address,Title,Description,Condition,Price,AdType,DateBegin,DateEnd,AllowEmail,ManagerName,ContactPhone,ImageNames,ImageUrls,Stock
0,6973130372467,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Ultramarines Venerable Dreadnought,ПРЕДЗАКАЗ. Релиз июль 2025!.\n\nJoyToy Warhamm...,Новое,7000,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/28/photo/01...,0
1,6973130372634,2689524122,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Blood Angels Furioso Dreadnought Samel,В пути!.\n\nJoyToy Warhammer 40K Blood Angels ...,Новое,8000,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,0
2,6973130372641,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Death Korps of Krieg Demolitions Specia...,В пути!.\n\nJoyToy Warhammer 40K Death Korps o...,Новое,2500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/A0...,0
3,6973130372658,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Death Korp of Krieg Veteran Squad Guard...,В пути!.\n\nJoyToy Warhammer 40K Death Korps o...,Новое,2500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/B0...,0
4,6973130372665,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Death Korps of Krieg Veteran Squad Serg...,В пути!.\n\nJoyToy Warhammer 40K Death Korps o...,Новое,2500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/24/photo/C0...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,76434,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 76434 Aragog in the Forbidden Forest,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 7...",Новое,2200,Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/blt6ffb...,1
171,21344,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 21344 The Orient Express Train,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 2...",Новое,28800,Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/blt4c79...,1
172,21350,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 21350 Jaws,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 2...",Новое,"20,000",Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/bltf513...,1
173,75429,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 75429 AT-AT Driver Helmet,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 7...",Новое,"8,500",Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/bltdd27...,1


In [ ]:
#file_path = r"C:\Users\Fibo\Проекты\WH40\avit_df.xlsx"

file_path = "/content/avit_df_final.xlsx"
avit_df_final.to_excel(file_path, float_format="%.0f", index=False)
# Загружаем файл на локальную машину
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##**Import XLSX to InSales**

In [ ]:
joytoy_wh_df

NameError: name 'joytoy_wh_df' is not defined

In [ ]:
#Выводим ключевые столбцы из парсинга датафрейма
insales_df = joytoy_wh_df[['Franchise', 'SKU (new)', 'Name', 'URL', 'Price_WAKO', 'Stock', 'Preorder', 'Applicable Age', 'Scale', 'Category']]

In [ ]:
# Фильтруем только джойтой
insales_df = insales_df[insales_df['Franchise'].isin(["JoyToy Warhammer 40K", "JoyToy Warhammer 30K"])]

In [ ]:
#value_list = ["6973130371897", "6973130372825", "6973130373327", "6973130373341", "6973130373785", "6973130373914", "6973130373921", "6973130373945", "6973130376281"]

# Фильтруем актуальные позиции
#insales_df = insales_df[~insales_df['SKU (new)'].isin(value_list)]

In [ ]:
insales_df

,Franchise,SKU (new),Name,URL,Price_WAKO,Stock,Preorder,Applicable Age,Scale,Category
0,JoyToy Warhammer 40K,6973130371286,Ultramarines Incursors Sergeant Romulo,https://img.joytoy.com/u_file/2206/22/photo/04...,2000,0,,15+,1:18,Ultramarines
1,JoyToy Warhammer 40K,6973130371287,Ultramarines Incursors Brother Varron,https://img.joytoy.com/u_file/2206/22/photo/02...,"2,000",1,,15+,1:18,Ultramarines
2,JoyToy Warhammer 40K,6973130371289,Ultramarines Incursors Brother Aurelian,https://img.joytoy.com/u_file/2206/22/photo/01...,"2,000",0,,15+,1:18,Ultramarines
3,JoyToy Warhammer 40K,6973130371288,Ultramarines Incursors Brother Remus,https://img.joytoy.com/u_file/2206/22/photo/03...,"2,000",0,,15+,1:18,Ultramarines
4,JoyToy Warhammer 40K,6973130371293,Ultramarines Infiltrators Brother Pullo,https://img.joytoy.com/u_file/2206/22/photo/02...,"2,000",0,,15+,1:18,Ultramarines
...,...,...,...,...,...,...,...,...,...,...
557,JoyToy Warhammer 40K,6927054400980,Black Legion Chosen Combi-melta and chainsword,https://img.joytoy.com/u_file/2501/02/photo/1-...,"4,600",0,ПРЕДЗАКАЗ. Релиз июнь 2025!,15+,1:18,Black Legion
558,JoyToy Warhammer 40K,6927054400997,Black Legion Chosen Champion with Bolter and P...,https://img.joytoy.com/u_file/2501/02/photo/1-...,"5,000",0,ПРЕДЗАКАЗ. Релиз июнь 2025!,15+,1:18,Black Legion
570,JoyToy Warhammer 30K,6927054401086,"Alpha Legion Alpharius,Primarch of the XXth Le...",https://img.joytoy.com/u_file/2502/06/photo/26...,"9,500",0,ПРЕДЗАКАЗ. Релиз май 2025!,15+,1:18,Alpha Legion
571,JoyToy Warhammer 30K,6927054401109,Alpha Legion Contemptor Dreadnought with Gravi...,https://img.joytoy.com/u_file/2502/06/photo/26...,"7,900",0,ПРЕДЗАКАЗ. Релиз май 2025!,15+,1:18,Alpha Legion


In [ ]:
# Выполняем переименование столбцов под требования шаблона
old_column_name = ['SKU (new)', 'Name', 'URL', 'Price_WAKO', 'Stock', 'Preorder'] #Текущие название столбцов
new_column_name = ['Артикул', 'Название товара или услуги', 'Изображения', 'Цена продажи', 'Остаток', 'Параметр: JOYTOY'] # Новые имена столбцов
insales_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)

<ipython-input-32-67e03c3545d8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insales_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)


In [ ]:
# Делаем описание для позиций В НАЛИЧИИ
Description_string_stock = """<p>В нaличии. Hовый, запечатанный.</p>
<p></p>
<p>{Franchise} {Название товара или услуги}</p>
<p></p>
<p>Oписание модeли:</p>
<p>· Маcштаб пpoдуктa: {Scale}</p>
<p>· Maтериал: АБС-ПВX</p>
<p>· Применимый вoзраcт: {Applicable Age}</p>
<p>· UРC : {Артикул}</p>
"""
Description_string_stock

'<p>В нaличии. Hовый, запечатанный.</p>\n<p></p>\n<p>{Franchise} {Название товара или услуги}</p>\n<p></p>\n<p>Oписание модeли:</p>\n<p>· Маcштаб пpoдуктa: {Scale}</p>\n<p>· Maтериал: АБС-ПВX</p>\n<p>· Применимый вoзраcт: {Applicable Age}</p>\n<p>· UРC : {Артикул}</p>\n'

In [ ]:
# Делаем описание для позиций - ПРЕДЗАКАЗ

Description_string_preorder = """<p>{Параметр: JOYTOY}</p>
<p></p>
<p>{Franchise} {Название товара или услуги}</p>
<p></p>
<p>Oписание модeли:</p>
<p>· Маcштаб пpoдуктa: {Scale}</p>
<p>· Maтериал: АБС-ПВX</p>
<p>· Применимый вoзраcт: {Applicable Age}</p>
<p>· UРC : {Артикул}</p>
<p></p>
<p>Добавьте ее в корзину и оплатите предзаказную стоимость.</p>
<p>Количество фигурок ограничено.</p>
<p>Ожидаемое время поступления уточняйте у менеджера.</p>

"""

Description_string_preorder

'<p>{Параметр: JOYTOY}</p>\n<p></p>\n<p>{Franchise} {Название товара или услуги}</p>\n<p></p>\n<p>Oписание модeли:</p>\n<p>· Маcштаб пpoдуктa: {Scale}</p>\n<p>· Maтериал: АБС-ПВX</p>\n<p>· Применимый вoзраcт: {Applicable Age}</p>\n<p>· UРC : {Артикул}</p>\n<p></p>\n<p>Добавьте ее в корзину и оплатите предзаказную стоимость.</p>\n<p>Количество фигурок ограничено.</p>\n<p>Ожидаемое время поступления уточняйте у менеджера.</p>\n\n'

In [ ]:
Description_string_nonstok = """<p>Наличие данной модели необходимо уточнить в ЛС.</p>
<p></p>
<p>{Franchise} {Название товара или услуги}</p>
<p></p>
<p>Oписание модeли:</p>
<p>· Маcштаб пpoдуктa: {Scale}</p>
<p>· Maтериал: АБС-ПВX</p>
<p>· Применимый вoзраcт: {Applicable Age}</p>
<p>· UРC : {Артикул}</p>
<p></p>
<p>Добавьте ее в корзину и оплатите предзаказную стоимость.</p>
<p>Количество фигурок ограничено.</p>
<p>Ожидаемое время поступления уточняйте у менеджера.</p>

"""
Description_string_nonstok

'<p>Наличие данной модели необходимо уточнить в ЛС.</p>\n<p></p>\n<p>{Franchise} {Название товара или услуги}</p>\n<p></p>\n<p>Oписание модeли:</p>\n<p>· Маcштаб пpoдуктa: {Scale}</p>\n<p>· Maтериал: АБС-ПВX</p>\n<p>· Применимый вoзраcт: {Applicable Age}</p>\n<p>· UРC : {Артикул}</p>\n<p></p>\n<p>Добавьте ее в корзину и оплатите предзаказную стоимость.</p>\n<p>Количество фигурок ограничено.</p>\n<p>Ожидаемое время поступления уточняйте у менеджера.</p>\n\n'

In [ ]:
def replace_placeholders(row):
    # Определяем шаблон в зависимости от нескольких условий
    if row['Остаток'] > '0':
        description = Description_string_stock
    elif row['Остаток'] == '0' and row['Параметр: JOYTOY'].strip() != '':
        description = Description_string_preorder
    else:
        description = Description_string_nonstok

    # Подставляем значения в выбранный шаблон
    description = description.replace('{Название товара или услуги}', row['Название товара или услуги'])
    description = description.replace('{Franchise}', row['Franchise'])
    description = description.replace('{Scale}', row['Scale'])
    description = description.replace('{Артикул}', str(row['Артикул']))
    description = description.replace('{Applicable Age}', row['Applicable Age'])
    description = description.replace('{Параметр: JOYTOY}', row['Параметр: JOYTOY'])

    return description

# Применение функции ко всем строкам DataFrame
insales_df['Краткое описание'] = insales_df.apply(replace_placeholders, axis=1)

In [ ]:
insales_df

,Franchise,Артикул,Название товара или услуги,Изображения,Цена продажи,Остаток,Параметр: JOYTOY,Applicable Age,Scale,Category,Краткое описание
0,JoyToy Warhammer 40K,6973130371286,Ultramarines Incursors Sergeant Romulo,https://img.joytoy.com/u_file/2206/22/photo/04...,2000,0,,15+,1:18,Ultramarines,<p>Наличие данной модели необходимо уточнить в...
1,JoyToy Warhammer 40K,6973130371287,Ultramarines Incursors Brother Varron,https://img.joytoy.com/u_file/2206/22/photo/02...,"2,000",1,,15+,1:18,Ultramarines,"<p>В нaличии. Hовый, запечатанный.</p>\n<p></p..."
2,JoyToy Warhammer 40K,6973130371289,Ultramarines Incursors Brother Aurelian,https://img.joytoy.com/u_file/2206/22/photo/01...,"2,000",0,,15+,1:18,Ultramarines,<p>Наличие данной модели необходимо уточнить в...
3,JoyToy Warhammer 40K,6973130371288,Ultramarines Incursors Brother Remus,https://img.joytoy.com/u_file/2206/22/photo/03...,"2,000",0,,15+,1:18,Ultramarines,<p>Наличие данной модели необходимо уточнить в...
4,JoyToy Warhammer 40K,6973130371293,Ultramarines Infiltrators Brother Pullo,https://img.joytoy.com/u_file/2206/22/photo/02...,"2,000",0,,15+,1:18,Ultramarines,<p>Наличие данной модели необходимо уточнить в...
...,...,...,...,...,...,...,...,...,...,...,...
557,JoyToy Warhammer 40K,6927054400980,Black Legion Chosen Combi-melta and chainsword,https://img.joytoy.com/u_file/2501/02/photo/1-...,"4,600",0,ПРЕДЗАКАЗ. Релиз июнь 2025!,15+,1:18,Black Legion,<p>ПРЕДЗАКАЗ. Релиз июнь 2025!</p>\n<p></p>\n<...
558,JoyToy Warhammer 40K,6927054400997,Black Legion Chosen Champion with Bolter and P...,https://img.joytoy.com/u_file/2501/02/photo/1-...,"5,000",0,ПРЕДЗАКАЗ. Релиз июнь 2025!,15+,1:18,Black Legion,<p>ПРЕДЗАКАЗ. Релиз июнь 2025!</p>\n<p></p>\n<...
570,JoyToy Warhammer 30K,6927054401086,"Alpha Legion Alpharius,Primarch of the XXth Le...",https://img.joytoy.com/u_file/2502/06/photo/26...,"9,500",0,ПРЕДЗАКАЗ. Релиз май 2025!,15+,1:18,Alpha Legion,<p>ПРЕДЗАКАЗ. Релиз май 2025!</p>\n<p></p>\n<p...
571,JoyToy Warhammer 30K,6927054401109,Alpha Legion Contemptor Dreadnought with Gravi...,https://img.joytoy.com/u_file/2502/06/photo/26...,"7,900",0,ПРЕДЗАКАЗ. Релиз май 2025!,15+,1:18,Alpha Legion,<p>ПРЕДЗАКАЗ. Релиз май 2025!</p>\n<p></p>\n<p...


In [ ]:
# Добавляем столбец с шаблонным описанием
insales_df['ID товара'] = ''
insales_df['Название товара в URL'] = ''
insales_df['URL'] = ''
insales_df['Полное описание'] = ''
insales_df['Видимость на витрине'] = 'выставлен'
insales_df['Применять скидки'] = 'Нет'
insales_df['Тег title'] = ''
insales_df['Мета-тег keywords'] = ''
insales_df['Мета-тег description'] = ''
insales_df['Размещение на сайте'] = 'Каталог ## Каталог/Joy Toy ## Каталог/Joy Toy/{Category}' ## Прописываем путь категории на сайте
insales_df['Весовой коэффициент'] = ''
insales_df['Валюта склада'] = 'RUR'
insales_df['НДС'] = 'Без НДС'
insales_df['Единица измерения'] = 'шт'
insales_df['Габариты'] = ''
insales_df['Ссылка на видео'] = ''
insales_df['ID варианта'] = ''
insales_df['Штрих-код'] = ''
insales_df['Внешний ID'] = ''
insales_df['Габариты варианта'] = ''
insales_df['Старая цена'] = ''
insales_df['Себестоимость'] = ''
insales_df['Вес'] = ''
insales_df['Изображения варианта'] = ''

In [ ]:
# Добавляем к пути каталога значение категорию из поля category
# description = description.replace('{Название товара или услуги}', row['Название товара или услуги'])
def replаce_category(row):
    # Определяем шаблон в зависимости от нескольких условий
    if row['Артикул'] != '':
      return row['Размещение на сайте'].replace('{Category}', row['Category'])
    else:
      return 'none'

# Применение функции
insales_df['Размещение на сайте'] = insales_df.apply(replаce_category, axis=1)

In [ ]:
#Указываем столбцы для вывода
columns_to_display = [
    'ID товара', 'Артикул', 'Название товара или услуги', 'Название товара в URL', 'URL', 'Краткое описание',
    'Полное описание', 'Видимость на витрине', 'Применять скидки', 'Тег title', 'Мета-тег keywords',
    'Мета-тег description', 'Размещение на сайте', 'Весовой коэффициент', 'Валюта склада', 'НДС',
    'Единица измерения', 'Габариты', 'Изображения', 'Ссылка на видео', 'ID варианта',
    'Штрих-код', 'Внешний ID', 'Габариты варианта', 'Цена продажи', 'Старая цена', 'Себестоимость',
    'Остаток', 'Вес', 'Изображения варианта', 'Параметр: JOYTOY'
]

# Выводим конкретные столбцы из DataFrame
insales_df = insales_df[columns_to_display]

In [ ]:
# Форматирование цены продажи - Удаляем запятую
insales_df['Цена продажи'] = insales_df['Цена продажи'].str.replace(',', '').astype(int)

<ipython-input-41-4daa4c928f32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insales_df['Цена продажи'] = insales_df['Цена продажи'].str.replace(',', '').astype(int)


In [ ]:
# Добавляем старую цену

# Значения для новых столбцов должны быть такими же, как в столбце 'A'
source_column = 'Цена продажи'

# Добавляем столбцы и заполняем их значениями из столбца 'A'
for column in columns_to_display:
    insales_df['Старая цена'] = insales_df[source_column]

<ipython-input-42-c29edc7bf004>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insales_df['Старая цена'] = insales_df[source_column]


In [ ]:
# Задаем специальное форматирование для фотографий в столбце "Изображения"
insales_df['Изображения'] = insales_df['Изображения'].replace(' \| ', ' ', regex=True)

<ipython-input-43-696c28e176f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insales_df['Изображения'] = insales_df['Изображения'].replace(' \| ', ' ', regex=True)


In [ ]:
# Замена нулей для предзаказов
insales_df.loc[insales_df['Параметр: JOYTOY'] != '', 'Остаток'] = (
    insales_df['Остаток'].replace(0, '10').astype(int)
)

In [ ]:
insales_df

,ID товара,Артикул,Название товара или услуги,Название товара в URL,URL,Краткое описание,Полное описание,Видимость на витрине,Применять скидки,Тег title,...,Штрих-код,Внешний ID,Габариты варианта,Цена продажи,Старая цена,Себестоимость,Остаток,Вес,Изображения варианта,Параметр: JOYTOY
0,,6973130371286,Ultramarines Incursors Sergeant Romulo,,,<p>Наличие данной модели необходимо уточнить в...,,выставлен,Нет,,...,,,,2000,2000,,0,,,
1,,6973130371287,Ultramarines Incursors Brother Varron,,,"<p>В нaличии. Hовый, запечатанный.</p>\n<p></p...",,выставлен,Нет,,...,,,,2000,2000,,1,,,
2,,6973130371289,Ultramarines Incursors Brother Aurelian,,,<p>Наличие данной модели необходимо уточнить в...,,выставлен,Нет,,...,,,,2000,2000,,0,,,
3,,6973130371288,Ultramarines Incursors Brother Remus,,,<p>Наличие данной модели необходимо уточнить в...,,выставлен,Нет,,...,,,,2000,2000,,0,,,
4,,6973130371293,Ultramarines Infiltrators Brother Pullo,,,<p>Наличие данной модели необходимо уточнить в...,,выставлен,Нет,,...,,,,2000,2000,,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,,6927054400980,Black Legion Chosen Combi-melta and chainsword,,,<p>ПРЕДЗАКАЗ. Релиз июнь 2025!</p>\n<p></p>\n<...,,выставлен,Нет,,...,,,,4600,4600,,0,,,ПРЕДЗАКАЗ. Релиз июнь 2025!
558,,6927054400997,Black Legion Chosen Champion with Bolter and P...,,,<p>ПРЕДЗАКАЗ. Релиз июнь 2025!</p>\n<p></p>\n<...,,выставлен,Нет,,...,,,,5000,5000,,0,,,ПРЕДЗАКАЗ. Релиз июнь 2025!
570,,6927054401086,"Alpha Legion Alpharius,Primarch of the XXth Le...",,,<p>ПРЕДЗАКАЗ. Релиз май 2025!</p>\n<p></p>\n<p...,,выставлен,Нет,,...,,,,9500,9500,,0,,,ПРЕДЗАКАЗ. Релиз май 2025!
571,,6927054401109,Alpha Legion Contemptor Dreadnought with Gravi...,,,<p>ПРЕДЗАКАЗ. Релиз май 2025!</p>\n<p></p>\n<p...,,выставлен,Нет,,...,,,,7900,7900,,0,,,ПРЕДЗАКАЗ. Релиз май 2025!


In [ ]:
#file_path = r"C:\Users\Fibo\Проекты\WH40\avit_df.xlsx"

#file_path = "/content/insales_df.xlsx"
#insales_df.to_excel(file_path, index=False)
# Загружаем файл на локальную машину
#files.download(file_path)

In [ ]:
gs = gspread.authorize(credentials)
worksheet = gs.open('waco_insales_import').worksheet('import_products')
print('таблица подключена')

таблица подключена


In [ ]:
# Запись датафрейма на ранее созданный лист с сохранением структуры датафрейма
set_with_dataframe(worksheet, insales_df)
format_with_dataframe(worksheet, insales_df, include_column_header=True)
print('Данные загружены в Google Таблицы')

Данные загружены в Google Таблицы


In [ ]:
# Получаем URL таблицы
#spreadsheet_url = worksheet.url
#print("URL созданной таблицы:", spreadsheet_url)

In [ ]:
#spreadsheet.share('lordyruss@gmail.com', perm_type='user', role='writer')

## YML import to VK

In [ ]:
avit_df_final

,Id,AvitoId,AdStatus,Category,GoodsType,VehicleType,Address,Title,Description,Condition,Price,AdType,DateBegin,DateEnd,AllowEmail,ManagerName,ContactPhone,ImageNames,ImageUrls,Stock
0,6973130372634,2689524122,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Blood Angels Furioso Dreadnought Samel,ПРЕДЗАКАЗ. Релиз февраль 2025!.\n\nJoyToy Warh...,Новое,8000,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,0
1,6973130372641,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Death Korps of Krieg Demolitions Specia...,ПРЕДЗАКАЗ. Релиз март 2025!.\n\nJoyToy Warhamm...,Новое,2800,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/A0...,0
2,6973130372658,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Death Korp of Krieg Veteran Squad Guard...,ПРЕДЗАКАЗ. Релиз март 2025!.\n\nJoyToy Warhamm...,Новое,2800,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/B0...,0
3,6973130372665,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Death Korps of Krieg Veteran Squad Serg...,ПРЕДЗАКАЗ. Релиз март 2025!.\n\nJoyToy Warhamm...,Новое,2800,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/24/photo/C0...,0
4,6973130372825,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Ultramarines Outriders Brother Catonus,ПРЕДЗАКАЗ. Релиз март 2025!.\n\nJoyToy Warhamm...,Новое,2800,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2411/27/photo/1-...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,WETA010,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",,Наличие данной модели необходимо уточнить в ЛС...,Новое,,Продаю своё,,,,,+7 985 182-56-36,,,
144,71045,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 71045 Minifigures Series 25,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 7...",Новое,550,Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/blt7dde...,15
145,75381,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 75381 Droideka,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 7...",Новое,7000,Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/bltd797...,1
146,21344,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Lego 21344 The Orient Express Train,"В нaличии. Hовый, запечатанный.\n\nLEGO Lego 2...",Новое,28800,Продаю своё,,,,,+7 985 182-56-36,,https://www.lego.com/cdn/cs/set/assets/blt4c79...,1


In [ ]:
# Под фильтрацию выборки (при необходимости)
# Фильтруем только джойтой
#avit_df_final = avit_df_final[avit_df_final['Franchise'].isin(["JoyToy Warhammer 40K", "JoyToy Warhammer 30K"])]

# Фильтруем датафрейм - убираем значения 'no data'
#vk_df = avit_df_final[insales_df['ID товара'].isin(["6927054400072", "6927054400065", "6927054400058", "6927054400102", "6927054400119", "6927054400089", "6927054400096", "6927054400126"])]

# Форматирование цены продажи - Удаляем запятую
#vk_df['Price'] = vk_df['Price'].str.replace(',', '').astype(int)

In [ ]:
# На основе датафрейма авито
vk_df = avit_df_final

# Фильтруем датафрейм - убираем значения 'no data'
vk_df = avit_df_final[avit_df_final['Id'].isin(["6927054400812", "6927054400805", "6927054400836", "6927054400843", "6927054400829", "6927054400867", "6927054400874", "6927054400850"])]

# Исключаем список позиций
#value_list = ["6927054400812", "6927054400805", "6927054400836", "6927054400843", "6927054400829", "6927054400867", "6927054400874", "6927054400850"]
#vk_df = vk_df[~vk_df['Id'].isin(value_list)]

vk_df

,Id,AvitoId,AdStatus,Category,GoodsType,VehicleType,Address,Title,Description,Condition,Price,AdType,DateBegin,DateEnd,AllowEmail,ManagerName,ContactPhone,ImageNames,ImageUrls,Stock
49,6927054400812,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Custodes Contemptor-Achillus Dreadnought,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,8700,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/20/photo/1-...,0
50,6927054400805,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Custodes Contemptor-Galatus Dreadnought,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,8700,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/20/photo/1....,0
52,6927054400836,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence Argent Lynx Vigilator 1,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
53,6927054400843,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence Argent Lynx Vigilator 2,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
54,6927054400829,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence Argent Lynx Knight,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,4300,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
55,6927054400867,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence White Falcon Vigilat...,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A-...,0
56,6927054400874,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence White Falcon Vigilat...,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
57,6927054400850,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence White Falcons Mistress,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0


In [ ]:
# На основе самого первого датафрейма из таблицы
vk_df = joytoy_wh_df

# Фильтруем датафрейм - убираем значения 'no data'
vk_df = avit_df_final[avit_df_final['Id'].isin(["6927054400812", "6927054400805", "6927054400836", "6927054400843", "6927054400829", "6927054400867", "6927054400874", "6927054400850"])]

# Исключаем список позиций
#value_list = ["6973130371897", "6973130372825", "6973130373327", "6973130373341", "6973130373785", "6973130373914", "6973130373921", "6973130373945", "6973130376281"]
#vk_df = vk_df[~vk_df['SKU (new)'].isin(value_list)]

vk_df

,Id,AvitoId,AdStatus,Category,GoodsType,VehicleType,Address,Title,Description,Condition,Price,AdType,DateBegin,DateEnd,AllowEmail,ManagerName,ContactPhone,ImageNames,ImageUrls,Stock
49,6927054400812,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Custodes Contemptor-Achillus Dreadnought,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,8700,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/20/photo/1-...,0
50,6927054400805,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Custodes Contemptor-Galatus Dreadnought,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,8700,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/20/photo/1....,0
52,6927054400836,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence Argent Lynx Vigilator 1,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
53,6927054400843,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence Argent Lynx Vigilator 2,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
54,6927054400829,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence Argent Lynx Knight,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,4300,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
55,6927054400867,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence White Falcon Vigilat...,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A-...,0
56,6927054400874,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence White Falcon Vigilat...,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0
57,6927054400850,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",JoyToy Sisters of Silence White Falcons Mistress,ПРЕДЗАКАЗ. Релиз июнь 2025!.\n\nJoyToy Warhamm...,Новое,3500,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2412/25/photo/A2...,0


In [ ]:
# Чистим лишние ссылки в фото. До 5 фотографий максимум
# Функция для оставления части строки по количеству вхождения символа
def keep_part_by_char_count(string, char, count):
    # Ищем индексы вхождений символа
    indices = [i for i, c in enumerate(string) if c == char]
    if len(indices) < count:
        return string
    else:
        return string[:indices[count-1]+1]

# Применяем функцию к определенным строкам в столбце 'text'
char = '|'
count = 5
vk_df['Link'] = vk_df['Link'].apply(lambda x: keep_part_by_char_count(x, char, count))
vk_df

KeyError: 'Link'

In [ ]:
# Шаблон XML
root = etree.Element("yml_catalog", date="2021-04-01 12:20")
shop = etree.SubElement(root, "shop")
name = etree.SubElement(shop, "name")
name.text = "JoyToy - Warhammer 40000, Dark Source, Infinity"
company = etree.SubElement(shop, "company")
company.text = "JoyToy - Warhammer 40000, Dark Source, Infinity"
url = etree.SubElement(shop, "url")
url.text = "https://vk.com/joytoy40k/"

currencies = etree.SubElement(shop, "currencies")
currency = etree.SubElement(currencies, "currency", id="RUB", rate="1")

categories = etree.SubElement(shop, "categories")
category1 = etree.SubElement(categories, "category", id="11")
category1.text = "Warhammer 40000"
category2 = etree.SubElement(categories, "category", id="12")
category2.text = "Warhammer 30K Horus Heresy"
category3 = etree.SubElement(categories, "category", id="13")
category3.text = "Dark Source"
category4 = etree.SubElement(categories, "category", id="14")
category4.text = "Infinity"
category5 = etree.SubElement(categories, "category", id="15")
category5.text = "Hardcore Coldplay"
category6 = etree.SubElement(categories, "category", id="16")
category6.text = "Dark Source-Jiang Hu"
category7 = etree.SubElement(categories, "category", id="17")
category7.text = "STRIFE"
category8 = etree.SubElement(categories, "category", id="18")
category8.text = "TMNT"
category9 = etree.SubElement(categories, "category", id="19")
category9.text = "В наличии"
category10 = etree.SubElement(categories, "category", id="20")
category10.text = "Предзаказ"
category11 = etree.SubElement(categories, "category", id="21")
category11.text = "Ultramarines"
category12 = etree.SubElement(categories, "category", id="22")
category12.text = "Blood Angels"
category13 = etree.SubElement(categories, "category", id="23")
category13.text = "Space Wolves"
category14 = etree.SubElement(categories, "category", id="24")
category14.text = "Chaos Space Marines"
category15 = etree.SubElement(categories, "category", id="25")
category15.text = "Imperial Fists"
category16 = etree.SubElement(categories, "category", id="26")
category16.text = "Astra Militarum"
category17 = etree.SubElement(categories, "category", id="27")
category17.text = "Black Templars"
category18 = etree.SubElement(categories, "category", id="28")
category18.text = "Orks"
category19 = etree.SubElement(categories, "category", id="29")
category19.text = "Grey Knights"
category20 = etree.SubElement(categories, "category", id="30")
category20.text = "T'au Empire"
category21 = etree.SubElement(categories, "category", id="31")
category21.text = "White Scars"
category22 = etree.SubElement(categories, "category", id="32")
category22.text = "Adepta Sororitas"
category23 = etree.SubElement(categories, "category", id="33")
category23.text = "Adeptus Custodes"
category24 = etree.SubElement(categories, "category", id="34")
category24.text = "Necrons"
category25 = etree.SubElement(categories, "category", id="35")
category25.text = "Raven Guard"
category26 = etree.SubElement(categories, "category", id="36")
category26.text = "Dark Angels"
category27 = etree.SubElement(categories, "category", id="37")
category27.text = "Salamanders"
category28 = etree.SubElement(categories, "category", id="38")
category28.text = "Iron Hands"
category29 = etree.SubElement(categories, "category", id="39")
category29.text = "White Consuls"
category30 = etree.SubElement(categories, "category", id="40")
category30.text = "Adeptus Mechanicus"
category31 = etree.SubElement(categories, "category", id="41")
category31.text = "Sons of Horus"
category32 = etree.SubElement(categories, "category", id="42")
category32.text = "Tyranids"
category33 = etree.SubElement(categories, "category", id="43")
category33.text = "Thousand Sons"

offers = etree.SubElement(shop, "offers")

In [ ]:
for _, row in vk_df.iterrows():  # менять датафрейм для выборки и без выборки - avit_df_final
    offer = etree.SubElement(offers, "offer", id=str(row['SKU (new)']), available="true" if row['Stock'] != '0' else "false")

    price = etree.SubElement(offer, "price")
    price.text = str(row['Price'])

    count = etree.SubElement(offer, "count")
    count.text = str(row['Stock'])

    # Добавление элемента categoryId с логикой наличия или предзаказа
    categoryId = etree.SubElement(offer, "categoryId")
    categoryId.text = str(row['Franchise'])

    # Добавление элемента categoryId с логикой наличия или предзаказа
    categoryId = etree.SubElement(offer, "categoryId")
    categoryId.text = "В наличии" if row['Stock'] != '0' else "Предзаказ"

    count.text = str(row['Stock'])
    currencyId = etree.SubElement(offer, "currencyId")
    currencyId.text = "RUB"
    name = etree.SubElement(offer, "name")
    name.text = row['']
    description = etree.SubElement(offer, "description")
    #description.text = 'Description'# для теста ставлю значение
    description.text = row['Description']# для теста ставлю значение
    image_urls = row['ImageUrls'].split(' | ')
    for image_url in image_urls:
        picture = etree.SubElement(offer, "picture")
        picture.text = image_url.strip()
        picture.text = image_url.replace(' |', '')
    # Добавление пустого тега <offers> после каждого <offer>
    offers = etree.SubElement(shop, "offers")

tree = etree.ElementTree(root)
tree.write("vk_df.xml", pretty_print=True, xml_declaration=True, encoding="utf-8")

In [ ]:
# Скачать файл на локальный компьютер
files.download("vk_df.xml")

In [ ]:
#file_path = r"C:\Users\Fibo\Проекты\WH40\avit_df.xlsx"

file_path = "/content/vk_df.xlsx"
avit_df_final.to_excel(file_path, float_format="%.0f", index=False)
# Загружаем файл на локальную машину
files.download(file_path)

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<yml_catalog date="2021-04-01 12:20">
  <shop>
    <name>JoyToy - Warhammer 40000, Dark Source, Infinity</name>
    <company>JoyToy - Warhammer 40000, Dark Source, Infinity</company>
    <url>https://vk.com/joytoy40k/</url>
    <currencies>
      <currency id="RUB" rate="1"/>
    </currencies>
    <categories>
      <category id="11">Warhammer 400001</category>
      <category id="2">Dark Source</category>
      <category id="3">В наличии</category>
      <category id="4">Предзаказ</category>
    </categories>
    <offers>
      <offer id="6973130379572" available="true">
        <price>3500</price>
        <count>0</count>
        <currencyId>RUB</currencyId>
        <name>Sons of Horus MKIV Sergeant with Power Fist</name>
        <description>Sons of Horus MKIV Sergeant with Power Fist</description>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7488.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7489.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7493.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7494.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7539.jpg</picture>
      </offer>

# LEGO

##**Import XLSX to Avito**

In [ ]:
joytoy_lego_df = joytoy_wh_df[joytoy_wh_df['Franchise'].str.contains('LEGO', case=False)]

In [ ]:
joytoy_lego_df

In [ ]:
#Создаем ключевые столбцы из парсинга датафрейма
lego_avito_df = joytoy_lego_df[['Franchise', 'SKU (new)', 'Applicable Age', 'Name', 'Link', 'URL', 'Stock', 'Price', 'Scale']]

In [ ]:
lego_avito_df

In [ ]:
Description_string = """

{Name}

Oписание модeли:
· Аpтикул: {SKU (new)}
· Boзрaст: {Applicable Age}

Oчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.

❤️ Добавь это объявление к себе в избранное

✅ В профиле есть еще очень много интересного!)

💬 Подписывайся на наши соцсети!

Если интересует что-то из LЕGО, Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!

# LЕGО, Лего, Конструктор, Набор,  lego star wars, lego marvel, lego city, lego heroes, lego ninjago, lego super heroes, lego technic, lego mindstorms, lego worlds, lego minecraft, lego harry potter, lego duplo, lego friends"""

Description_string

In [ ]:
lego_avito_df.loc[:, ['AvitoId', 'AdStatus', 'Category', 'GoodsType', 'VehicleType', 'Address', 'Condition', 'AdType', 'ContactPhone', 'DateBegin', 'DateEnd', 'AllowEmail', 'ManagerName', 'ImageNames', 'Description']] = [
    '', 'Free', 'Коллекционирование', 'Модели', '', 'Москва, метро Улица Скобелевская', 'Новое', 'Продаю своё', '+7 985 182-56-36', '', '', '', '', '', Description_string
]

In [ ]:
lego_avito_df

In [ ]:
# заменяем в столбце Description заглушку {name} на значение товара из столбца name
def replace_placeholders(row):
  description = row['Description']
  description = description.replace('{Name}', row['Name'])
  #description = description.replace('{Franchise}', row['Franchise'])
  description = description.replace('{Applicable Age}', row['Applicable Age'])
  description = description.replace('{SKU (new)}', row['SKU (new)'])
  return description

# Применение функции с использованием .loc[] для присвоения значений
lego_avito_df.loc[:, 'Description'] = lego_avito_df.apply(replace_placeholders, axis=1)

In [ ]:
lego_avito_df

In [ ]:
# Выполняем переименование столбцов под требования шаблона
new_column_name = ['Id', 'Title', 'ImageUrls', 'Stock']
old_column_name = ['SKU (new)', 'Name', 'URL', 'В наличии'] # имя столбца, которое вы хотите изменить
lego_avito_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)

In [ ]:
# Выводим столбцы в порядке шаблона
lego_avito_df_final = lego_avito_df[['Id', 'AvitoId', 'AdStatus', 'Category', 'GoodsType', 'VehicleType', 'Address', 'Title', 'Description', 'Condition', 'Price', 'AdType', 'DateBegin', 'DateEnd', 'AllowEmail', 'ManagerName', 'ContactPhone', 'ImageNames', 'ImageUrls', 'Stock']]
lego_avito_df_final

In [ ]:
#file_path = r"C:\Users\Fibo\Проекты\WH40\avit_df.xlsx"

file_path = "/content/lego_avito_df_final.xlsx"
lego_avito_df_final.to_excel(file_path, index=False)
# Загружаем файл на локальную машину
files.download(file_path)